In [72]:
# Imports
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from dotenv import load_dotenv
from helpers import get_database_url
load_dotenv()
import os

In [73]:
# Create dataframes from csv folder within the same folder
messages = pd.read_csv("csvs/messages.csv", index_col=0)
categories = pd.read_csv("csvs/categories.csv", index_col=0)

In [74]:
# Merge the dataframe
df = messages.merge(categories, on="id")

<h2>Separate category data </h2>

In [76]:
categories = categories["categories"].str.split(";", expand=True)

In [77]:
get_name = lambda x:x.split('-')[0]

In [78]:
get_number = lambda x:x.split('-')[1]

In [79]:
category_columns = list(categories.iloc[0].apply(get_name))

In [81]:
categories.columns = category_columns

In [82]:
for column in categories:
    categories[column] = categories[column].apply(get_number)
    
    categories[column] = pd.to_numeric(categories[column])
    

In [83]:
# Confirm the columns are numeric
categories["related"].dtype

dtype('int64')

<h2> Remove old categories column and join the df and category dataframes </h2>

In [84]:
df.drop(["categories"], axis = 1, inplace=True)

In [85]:
df = df.join(categories, on="id", how="outer")

In [86]:
df.shape

(26668, 39)

In [87]:
df.duplicated().sum()

453

In [88]:
# We expect 26,177 rows
df.drop_duplicates(inplace=True)

<h2> Add the data to a database </h2>

In [90]:
url = get_database_url()

In [91]:
engine = create_engine(url)
df.to_sql('message_categories', engine, index=False, if_exists='replace')